In [1]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate

In [13]:
import pandas as pd
from langchain_openai import OpenAI

df = pd.read_parquet('./data/parquet/sources-camel-kafka.parquet', engine='pyarrow')
df.head()

,project,path,language,tags,description,source
0,camel,/Users/opiske/code/java/camel/components/camel...,java,,,/*\n * Licensed to the Apache Software Foundat...
1,camel,/Users/opiske/code/java/camel/components/camel...,java,,,/*\n * Licensed to the Apache Software Foundat...
2,camel,/Users/opiske/code/java/camel/components/camel...,java,,,/*\n * Licensed to the Apache Software Foundat...
3,camel,/Users/opiske/code/java/camel/components/camel...,java,,,/*\n * Licensed to the Apache Software Foundat...
4,camel,/Users/opiske/code/java/camel/components/camel...,java,,,/*\n * Licensed to the Apache Software Foundat...


In [19]:
from langchain_community.document_loaders import DataFrameLoader
loader = DataFrameLoader(df, page_content_column="source")
docs = loader.load()
print("Num docs: ", len(docs))

Num docs:  12


In [25]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the sources found in the paqrquet files.
retriever = vectorstore.as_retriever()
prompt = PromptTemplate.from_template("""
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know.     

    Question: {question} 

    Context: {context} 

    Answer:
""")

llm = ChatOpenAI(model_name="gpt-3.5-turbo-1106", temperature=0)

def format_docs(docs):    
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#result = rag_chain.invoke("Show me a Java example of processAsyncSendsMessageWithException in Kafka")
#print(result)

result = rag_chain.invoke("How can I process exception during an async send in Kafka? Can you show me an example in Java?")
print(result)


To process an exception during an async send in Kafka in Java, you can use the Mockito framework to mock the Kafka producer and throw an exception when the send method is called. Here's an example:

```java
@Test
public void processAsyncSendsMessageWithException() {
    endpoint.getConfiguration().setTopic("sometopic");
    Mockito.when(exchange.getIn()).thenReturn(in);
    Mockito.when(exchange.getMessage()).thenReturn(in);

    // set up the exception here
    org.apache.kafka.clients.producer.Producer kp = producer.getKafkaProducer();
    Mockito.when(kp.send(any(ProducerRecord.class), any(Callback.class))).thenThrow(new ApiException());

    in.setHeader(KafkaConstants.PARTITION_KEY, 4);

    producer.process(exchange, callback);
}
```

In this example, the `processAsyncSendsMessageWithException` test method sets up the mock behavior using Mockito to throw an `ApiException` when the `send` method of the Kafka producer is called. This allows you to simulate and handle the exception 